In [11]:
import pandas as pd
pd.options.display.max_columns=100
import kaggle
import pickle

import os
path=r"C:\Users\berid\python\yts mx project"

In [12]:
yts_df=pd.read_csv(os.path.join(path,'scraped_data','cleaned_yts_movies_df.csv'))
letterboxd_df=pd.read_csv(os.path.join(path,'scraped_data','cleaned_letterboxd_movies_df.csv'))

##### merging dataframes

In [13]:
df=yts_df.merge(letterboxd_df,how='left',on='IMDB_LINK')

In [14]:
df['GENRES_x']=df['GENRES_x']+'|'
df['DIRECTOR_x']=df['DIRECTOR_x']+'|'
df['BIGRAMS_TAGS_x']=df['BIGRAMS_TAGS_x']+'|'
df['DIRECTOR_y']=df['DIRECTOR_y']+'|'
df['BIGRAMS_TAGS_y']=df['BIGRAMS_TAGS_y']+'|'

##### applying weights

In [15]:
# Define weights
weights = {
    'GENRES_x': 12,
    'DIRECTOR_x': 5,
    'TAGS_x': 3,
    'BIGRAMS_TAGS_x': 1,
    'GENRES_y': 1,
    'DIRECTOR_y': 5,
    'TAGS_y': 10,
    'BIGRAMS_TAGS_y': 1
}

df['ALL_TAGS'] = df.apply(
    lambda row: ''.join(
        str(row[col]) * weight for col, weight in weights.items()
        if pd.notnull(row[col])
    ),
    axis=1
)

df['ALL_TAGS'] = df['ALL_TAGS'].str.replace('None', '').str.replace('nan', '').str.replace('Show All…', '|').str.replace('||', '|')


In [16]:
df=df[['IMDB_LINK','POSTER','TITLE_x','YEAR_x','GENRES_x','DIRECTOR_x','IMDB_RATING','IMDB_RATING_COUNT','ALL_TAGS']]
df.columns=df.columns.str.replace('_x','')

In [17]:
df.to_csv(os.path.join(path,'scraped_data','merged_df.csv'),index=False)

##### create similarity matrix

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(tokenizer=lambda x: x.split('|'), stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['ALL_TAGS'])

similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

C:\Users\berid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
import numpy as np
import pickle

num_parts = 20
array_parts = np.array_split(similarity_matrix, num_parts)

for i, part in enumerate(array_parts):
    with open(os.path.join(path,'scraped_data','similarities_matrix',f'array_part_{i}.pkl'), "wb") as f:
        pickle.dump(part, f)

In [20]:
df['ALL_TAGS']

0       Action|Crime|Drama|Action|Crime|Drama|Action|C...
1       Action|Adventure|Crime|Drama|Thriller|Action|A...
2       Action|Adventure|Crime|Mystery|Sci-Fi|Thriller...
3       Action|Drama|Action|Drama|Action|Drama|Action|...
4       Action|Comedy|Drama|Romance|Action|Comedy|Dram...
                              ...                        
7446    Action|Crime|Drama|Horror|Mystery|Thriller|Act...
7447    Action|Drama|Musical|Romance|Action|Drama|Musi...
7448    Action|Crime|Drama|Music|Action|Crime|Drama|Mu...
7449    Comedy|Drama|Family|Sport|Comedy|Drama|Family|...
7450    Action|Comedy|Drama|Romance|Action|Comedy|Dram...
Name: ALL_TAGS, Length: 7451, dtype: object